In [ ]:
# Cài đặt các thư viện cần thiết (nếu chưa có)
!pip install requests beautifulsoup4 pandas numpy torch
!pip install -U sentence-transformers faiss-cpu selenium
!apt-get update
!apt-get install -y chromium-chromedriver

# Cấu hình cho Selenium trong Colab
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
# Import thư viện
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import time
from sentence_transformers import SentenceTransformer
import faiss
import pickle
from tqdm.notebook import tqdm
import json
from selenium.webdriver.common.by import By

# Khởi tạo driver cho Colab
driver = webdriver.Chrome(options=options)

print("Tất cả thư viện đã được import và Selenium đã được cấu hình!")

In [ ]:
# Xây dựng bộ crawler dữ liệu từ Tiki

def get_product_links(category_url):
    """Lấy link của các sản phẩm từ trang danh mục Tiki."""
    driver.get(category_url)
    time.sleep(5) # Chờ trang tải JavaScript
    
    # Lăn chuột xuống để tải thêm sản phẩm
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(2): # Lăn 2 lần
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    product_links = []
    # Tiki sử dụng thẻ 'a' với attribute 'href' cho sản phẩm
    for a in soup.find_all('a', href=True):
        href = a['href']
        if '.html' in href and 'p' in href and 'spid' not in href:
            if not href.startswith('https://tiki.vn'):
                href = 'https://tiki.vn' + href
            product_links.append(href)
    return list(set(product_links)) # Loại bỏ link trùng lặp

def scrape_product_details(product_url):
    """Crawl thông tin chi tiết của một sản phẩm."""
    try:
        driver.get(product_url)
        time.sleep(3) # Chờ tải
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Tên sản phẩm
        name = soup.find('h1', class_='title').get_text(strip=True) if soup.find('h1', class_='title') else "N/A"

        # Giá
        price_str = soup.find('div', class_='product-price__current-price').get_text(strip=True) if soup.find('div', class_='product-price__current-price') else "0"
        price = int(re.sub(r'[^\d]', '', price_str))
        
        # Mô tả
        description_div = soup.find('div', {'data-testid': 'desc-content'})
        description = description_div.get_text(strip=True, separator='\n') if description_div else ""

        # Thông số kỹ thuật
        specs = {}
        spec_table = soup.find('table', class_='Content_table')
        if spec_table:
            rows = spec_table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    key = cells[0].get_text(strip=True)
                    value = cells[1].get_text(strip=True)
                    specs[key] = value

        return {
            'url': product_url,
            'name': name,
            'price': price,
            'description': description,
            'specifications': specs
        }
    except Exception as e:
        print(f"Lỗi khi crawl sản phẩm {product_url}: {e}")
        return None

# === Thực thi crawling ===
CATEGORY_URL = "https://tiki.vn/laptop/c1846"
print(f"Bắt đầu crawl link sản phẩm từ: {CATEGORY_URL}")
product_links = get_product_links(CATEGORY_URL)
print(f"Tìm thấy {len(product_links)} link sản phẩm.")

# Giới hạn số lượng để demo
product_links = product_links[:20]

all_products_data = []
print("\nBắt đầu crawl thông tin chi tiết từng sản phẩm...")
for link in tqdm(product_links):
    product_data = scrape_product_details(link)
    if product_data and product_data['name'] != "N/A":
        all_products_data.append(product_data)

driver.quit() # Đóng trình duyệt
print(f"\nHoàn thành! Crawl được {len(all_products_data)} sản phẩm.")

df_ecommerce = pd.DataFrame(all_products_data)
df_ecommerce.head()

In [ ]:
# Tạo trường văn bản tổng hợp để vector hóa

def create_searchable_text(row):
    """Tạo một chuỗi văn bản duy nhất chứa các thông tin quan trọng nhất của sản phẩm."""
    name = row['name']
    
    # Chuyển đổi dict thông số kỹ thuật thành chuỗi
    specs_list = []
    if isinstance(row['specifications'], dict):
        for key, value in row['specifications'].items():
            specs_list.append(f"{key}: {value}")
    specs_text = ". ".join(specs_list)
    
    # Ghép nối các thông tin
    # Đây là bước quan trọng để tạo ngữ cảnh cho việc tìm kiếm
    # Ví dụ: "Laptop ABC, chip Intel Core i5, RAM 8GB. Đây là dòng laptop văn phòng..."
    searchable = f"Tên sản phẩm: {name}. Thông số: {specs_text}. Mô tả: {row['description'][:300]}"
    return searchable

df_ecommerce['searchable_text'] = df_ecommerce.apply(create_searchable_text, axis=1)

# Lưu KB dạng thô
df_ecommerce.to_json('ecommerce_kb.json', orient='records', lines=True, force_ascii=False)

df_ecommerce[['name', 'searchable_text']].head()

In [ ]:
# Vector hóa và xây dựng chỉ mục tìm kiếm FAISS

# Sử dụng lại model đã khai báo ở phần Healthcare
# Nếu chưa có, hãy chạy lại cell tải model
print("Bắt đầu tải model Sentence Transformer (nếu chưa có)...")
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')
print("Model đã sẵn sàng.")

chunks_to_encode = df_ecommerce['searchable_text'].tolist()

print(f"Bắt đầu vector hóa {len(chunks_to_encode)} sản phẩm...")
embeddings = model.encode(chunks_to_encode, show_progress_bar=True, normalize_embeddings=True)
print("Vector hóa hoàn tất!")

# Xây dựng chỉ mục FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index = faiss.IndexIDMap(index)
ids = np.array(df_ecommerce.index).astype('int64') # Dùng index của DataFrame làm ID
index.add_with_ids(embeddings, ids)

print(f"Đã thêm {index.ntotal} vector vào chỉ mục FAISS.")

# Lưu trữ
# 1. Dữ liệu gốc đã được lưu ở file ecommerce_kb.json
# 2. Lưu chỉ mục FAISS
with open('ecommerce_faiss.index', 'wb') as f:
    faiss.write_index(index, faiss.StandardIOWriter(f))

print("Chỉ mục FAISS cho sản phẩm đã được lưu.")

In [ ]:
# Cell 6: Demo tìm kiếm sản phẩm trong Knowledge Base

# Load lại KB và chỉ mục
df_kb_loaded = pd.read_json('ecommerce_kb.json', orient='records', lines=True)
with open('ecommerce_faiss.index', 'rb') as f:
    index_loaded = faiss.read_index(faiss.StandardIOWriter(f))
    
model_loaded = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

def search_products(query, top_k=3):
    """Hàm tìm kiếm sản phẩm trong KB."""
    print(f"Truy vấn của bạn: '{query}'")
    
    query_embedding = model_loaded.encode([query], normalize_embeddings=True)
    distances, ids = index_loaded.search(query_embedding, top_k)
    
    results = []
    print("\n--- Các sản phẩm phù hợp nhất ---")
    for i, doc_id in enumerate(ids[0]):
        if doc_id != -1:
            product = df_kb_loaded.iloc[doc_id]
            score = 1 - distances[0][i]
            
            print(f"\nKết quả {i+1} (Score: {score:.4f})")
            print(f"   Tên: {product['name']}")
            print(f"   Giá: {product['price']:,}đ")
            print(f"   URL: {product['url']}")
            
            # In ra một vài thông số chính
            specs = product.get('specifications', {})
            if isinstance(specs, dict):
                cpu = specs.get('CPU', 'N/A')
                ram = specs.get('RAM', 'N/A')
                print(f"   CPU: {cpu}, RAM: {ram}")
            
            results.append(product.to_dict())
            
    return results

# Thử tìm kiếm
search_products("laptop mỏng nhẹ cho sinh viên")
print("\n" + "="*50 + "\n")
search_products("máy tính gaming có card đồ họa rời")